In [ ]:
%matplotlib notebook

In [ ]:
# loading standard modules
import numpy as np
import matplotlib.pyplot as plt
import maxflow
import math

from scipy import ndimage, signal
from skimage import img_as_ubyte
from skimage.color import rgb2grey
# loading custom module (requires file asg1.py in the same directory as the notebook file)
from asg1 import Figure, GraphCutsPresenter

In [ ]:
help(maxflow)

In [ ]:
class MyGraphCuts:
    bgr_value = 0
    obj_value = 1
    none_value = 2
    
    def __init__(self, img):
        self.fig = Figure()
        self.pres = GraphCutsPresenter(img, self)
        self.pres.connect_figure(self.fig)
        
        self.num_rows = img.shape[0]
        self.num_cols = img.shape[1]
        
        
        # build image-contrast weighted edge grid
        dx = np.array([[ 0.0, 0.0, 0.0],
                       [ 1.0, 0.0,-1.0],
                       [ 0.0, 0.0, 0.0]])

        dy = np.array([[ 0.0,-1.0, 0.0],
                       [ 0.0, 0.0, 0.0],
                       [ 0.0, 1.0, 0.0]])
        
        im_blurred = ndimage.gaussian_filter(img[:,:], sigma=1)
        im_dx = signal.convolve2d(im_blurred, dx, boundary='symm', mode='same')
        im_dy = signal.convolve2d(im_blurred, dy, boundary='symm', mode='same')
        self.weight_grid = np.sqrt(im_dx**2 + im_dy**2)
        
        # 4-neighbors graph
        self.neighbor_xs = [0, 1]
        self.neighbor_ys = [1, 0]

        # 8-neighbors graph
        #self.neighbor_xs = [0, 1, 1,  1,  0, -1, -1, -1]
        #self.neighbor_ys = [1, 1, 0, -1, -1, -1,  0,  1]

    def run(self):
        self.fig.show()

    def compute_labels(self, seed_mask):   
        num_rows = self.num_rows
        num_cols = self.num_cols
        
        weight_grid = self.weight_grid
        neighbor_xs = self.neighbor_xs
        neighbor_ys = self.neighbor_ys
        num_neighbors = len(neighbor_xs)
        
        # +---------+---------+
        # |         |         |
        # |   bgr   |  none   |
        # |         |         |
        # +---------+---------+
        # |         |         |
        # |  none   |   obj   |
        # |         |         |
        # +---------+---------+
        label_mask = np.full((num_rows, num_cols), self.none_value, dtype='uint8')
        label_mask[:num_rows // 2, :num_cols // 2] = self.bgr_value
        label_mask[num_rows // 2:, num_cols // 2:] = self.obj_value
        
        g = maxflow.Graph[int](num_rows*num_cols, num_rows*num_cols)
        g.add_nodes(num_rows*num_cols)
        
        # temp value
        sigma = 1
        
        node = 0
        node_d = {}
        # find all seeds and connect them to appropriate terminal/neighbors
        for i in range(num_rows):
            for j in range(num_cols):
                if (i,j) not in node_d:
                    node_d[(i,j)] = node++    
                
                val = seed_mask[num_rows, num_cols]
                
                if (val == bgr_value or val == obj_value):
                    # add "prohibitively expensive" terminal edge
                    g.add_tedge(node_d[(i,j)], 0, 0)
                else
                    # connect edge with neighbors
                    for k in xrange(num_neighbors):
                        nx = x + neighbor_xs[k]
                        ny = y + neighbor_ys[k]
                        
                        if 0 <= nx and nx < num_cols and 0 <= ny and ny < num_rows:
                            if (nx, ny) not in node_d
                                node_d[(nx, ny)] = node++
                            # TODO: calculate edge weight
                            g.add_edge(node_d[(i,j)], node_d[(nx, ny)], 0, 0)
         
        # source and sink
        #g.add_tedge(0, 0, 0)
        #g.add_tedge(0, 0 ,0)
        
        # calculate max flow
        #g.maxflow()
        
        #g.get_segments()
        
        return label_mask

### Notes about the basic graph cut interface:
1. To provide the regional hard constraints (seeds) for object and background segments use left and right mouse clicks (mouse dragging works somewhat too). Use mouse wheel to change the brush size.
2. The seed mask is built by the "GraphCutsPresenter". Each mouse release activates "on_mouse_up" function of the presenter, which asks the linked MyGraphCuts object to "compute_labels" for all pixels
based on the provided seed mask.
3. You should use "PyMaxflow" library (already imported into this notebook if you ran the second cell) to build a weighted graph and to compute a minimum s/t cut defining all pixel labels from the seeds as explain in topic 5.

In [ ]:
img = plt.imread('images/canada.bmp')
app = MyGraphCuts(img[:800,:800])
app.run()